<a href="https://colab.research.google.com/github/HSV-AI/presentations/blob/master/2021/210127_WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

This notebook is based of the LDA notebook in the [Bug Analysis](https://github.com/HSV-AI/bug-analysis/blob/master/notebooks/Topic_Modeling_LDA.ipynb) project. It has been reduced to only include the necessary items to demonstrate the integration of Weights and Biases.

## Global Imports

In [6]:
from datetime import date
from datetime import time
from datetime import datetime
from time import sleep
from math import ceil
from dateutil import parser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA


# Loading Bugzilla Data

In [7]:
df = pd.read_parquet('https://github.com/HSV-AI/bug-analysis/blob/master/data/df-xtext.parquet.gzip?raw=true')

# Weights and Biases Install and Login

This was copied from the WandB [quickstart](https://docs.wandb.ai/quickstart)



In [4]:
%%capture
!pip install wandb -qqq
import wandb

Since we will be running this in an interactive notebook fashion, we will use the reinit=True in order to tell wandb that we are beginning a new run. This is paired with a run.finish() later in the notebook to complete the run. All logging between the init and the finish are associated with the run.

In [8]:
run = wandb.init(project="topic-modeling-lda", reinit=True)

wandb: WARNING Ignored wandb.init() arg project when running a sweep


# Create a CountVectorizer

The parameters that we are investigating w.r.t. the CountVectorizer are the max and min document frequence. Throw out any words that occur in documents less than the minimum frequency or more than the maximum frequency.

In [9]:
vectorizer_params = { 'max_df':0.8, 'min_df':5}

wandb.log(vectorizer_params)
tf_vectorizer = CountVectorizer(**vectorizer_params)
tf = tf_vectorizer.fit_transform(df.loc[:,'text'])
tf_feature_names = tf_vectorizer.get_feature_names()
# Materialize the sparse data
dense = tf.todense()

data_sparsicity = ((dense > 0).sum()/dense.size)*100
wandb.log({'data_sparsicity':data_sparsicity})
# Compute Sparsicity = Percentage of Non-Zero cells
print("Data Sparsicity:", data_sparsicity, "%")
print("Stop Word Count:",len(tf_vectorizer.stop_words_))

Data Sparsicity: 1.2288689104602475 %
Stop Word Count: 35588


# LDA

For LDA, check the perplexity. The lower the better. The score will be a negative number. The closer to 0 the better.

In [10]:
lda_params = {
    'n_components':15,
    'max_iter':5,
    'learning_offset':50
}

wandb.log(lda_params)

# Add constant params
lda_params['learning_method'] = 'online'
lda_params['random_state'] = 0

# Run LDA
lda = LatentDirichletAllocation(**lda_params).fit(tf)

score = lda.score(tf)
perplexity = lda.perplexity(tf)

wandb.log({
    'score':score,
    'perplexity':perplexity
})
print("\nLog Likelihood: %d ## Perplexity: %d\n" % ((lda.score(tf)), lda.perplexity(tf)))



Log Likelihood: -11152386 ## Perplexity: 502



# Visualize the Results

This section uses several methods to visualize the results of the LDA.

Using PCA, create a 2D Visualizaton of the TF/IDF Matrix.

In [11]:
# Print the top 5 words in each topic
num_words = 5
print('LDA Topics: ')
for topic_idx, topic in enumerate(lda.components_):
    print ("Topic %d:" % (topic_idx), " ".join([tf_feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]))

LDA Topics: 
Topic 0: typesystem xbase xtext rawtypeconformancecomputer intellij
Topic 1: javareconcilingstrategy projectconfigurationmanager equinoxbundle mavenexecutioncontext moduleclassloader
Topic 2: e4 partrenderingengine workbench swt ui
Topic 3: outline formatting editor formatter jvmmodel
Topic 4: org eclipse xtext http from
Topic 5: com google inject injectorimpl elements
Topic 6: ide guava shared jetty awt
Topic 7: line object thread string run
Topic 8: refactoring deltaconverter storage2urimapperjavaimpl deltaprocessor resource
Topic 9: is that and of this
Topic 10: java at eclipse org internal
Topic 11: xtext org java eclipse common
Topic 12: buildmanager builder xbase events xtextbuilder
Topic 13: emf generator mwe2 antlr mwe
Topic 14: may of and https your


In [12]:

X = tf.todense()

pca = PCA(n_components=2).fit(X)
data2D = pca.transform(X)
plt.scatter(data2D[:,0], data2D[:,1])
plt.plot()

wandb.log({'PCA Scatter': plt})

In [13]:
run.finish()

max_df,0.8
min_df,5
_step,4
_runtime,222
_timestamp,1611898184
data_sparsicity,1.22887
n_components,15
max_iter,5
learning_offset,50
score,-11152386.41522
perplexity,502.95997


max_df,▁
min_df,▁
_step,▁▃▅▆█
_runtime,▁▁▃▄█
_timestamp,▁▁▃▄█
data_sparsicity,▁
n_components,▁
max_iter,▁
learning_offset,▁
score,▁
perplexity,▁


# Clean Sweep!

We can also use wandb to create a sweep with a set of parameters and a method for searching.

In order to use the sweep, everything needs to be in a single method (can't reference items outside of the method) and then the method passed to the agent. So far, it appears that this will continue running until you stop it.

There is probably a way to have it auto stop after the metric meets some criteria.

In [5]:
sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "score", "goal": "maximize"},
    "parameters": {
        "max_df": {"min":0.5, "max":0.95 },
        "min_df": {"min":.05, "max":0.5 },
        "n_components": {"min":5, "max":20},
        "max_iter": {"min":1, "max":10}
    },
}

sweep_id = wandb.sweep(sweep_config, project="Test Sweep")

def sweep():

    # Initialize a new wandb run
    wandb.init()

    df = pd.read_parquet('https://github.com/HSV-AI/bug-analysis/blob/master/data/df-xtext.parquet.gzip?raw=true')
    
    # The Sweep parameters are passed in with the wandb.config parameter

    tf_vectorizer = CountVectorizer(min_df=wandb.config['min_df'], max_df=wandb.config['max_df'])

    tf = tf_vectorizer.fit_transform(df.loc[:,'text'])
    tf_feature_names = tf_vectorizer.get_feature_names()
    # Materialize the sparse data
    dense = tf.todense()

    data_sparsicity = ((dense > 0).sum()/dense.size)*100
    wandb.log({'data_sparsicity':data_sparsicity})
    # Compute Sparsicity = Percentage of Non-Zero cells
    print("Data Sparsicity:", data_sparsicity, "%")
    print("Stop Word Count:",len(tf_vectorizer.stop_words_))

    # Run LDA
    lda = LatentDirichletAllocation(n_components=wandb.config['n_components'], max_iter=wandb.config['max_iter']).fit(tf)

    score = lda.score(tf)
    perplexity = lda.perplexity(tf)

    wandb.log({
        'score':score,
        'perplexity':perplexity
    })

    # Sync wandb
    wandb.join()

wandb.agent(sweep_id, sweep)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: b7hdqox8
Sweep URL: https://wandb.ai/jperiodlangley/Test%20Sweep/sweeps/b7hdqox8


wandb: Agent Starting Run: q9aiqtro with config:
wandb: 	max_df: 0.6405019445690965
wandb: 	max_iter: 8
wandb: 	min_df: 0.18924901714257797
wandb: 	n_components: 20
wandb: Currently logged in as: jperiodlangley (use `wandb login --relogin` to force relogin)


Data Sparsicity: 30.38919136922767 %
Stop Word Count: 44992


...

wandb: Agent Starting Run: tci00suh with config:
wandb: 	max_df: 0.9090411180333067
wandb: 	max_iter: 10
wandb: 	min_df: 0.20919597458504746
wandb: 	n_components: 6


Data Sparsicity: 36.88296398891967 %
Stop Word Count: 44999


data_sparsicity,36.88296
_step,1
_runtime,48
_timestamp,1611896889
score,-2364718.36836
perplexity,25.72041


data_sparsicity,▁
_step,▁█
_runtime,▁█
_timestamp,▁█
score,▁
perplexity,▁


wandb: Agent Starting Run: kthlule8 with config:
wandb: 	max_df: 0.7893123423826052
wandb: 	max_iter: 7
wandb: 	min_df: 0.4744416368429804
wandb: 	n_components: 17
wandb: Ctrl + C detected. Stopping sweep.
